In [2]:
pip install yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.4 MB/s eta 0:00:00


In [3]:
import yt_dlp
import subprocess
import os

def download_video(url, save_path="."):
    ydl_opts = {
        'outtmpl': f'{save_path}/%(title)s.%(ext)s',
        'format': 'best',  # Download best combined stream
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        filename = ydl.prepare_filename(info)
        return filename

def convert_quality(input_file, resolution, output_file):
    # FFmpeg command to downscale
    subprocess.run([
        "ffmpeg", "-i", input_file,
        "-vf", f"scale=-2:{resolution}",
        "-c:a", "copy",  # Keep original audio
        output_file
    ], check=True)

if __name__ == "__main__":
    video_url = input("Enter YouTube video URL: ").strip()
    save_directory = input("Enter save directory (leave empty for current folder): ").strip() or "."

    # Step 1: Download original video
    print("Downloading original video...")
    original_file = download_video(video_url, save_directory)

    # Step 2: Convert to low and ultra-low quality
    ultra_low_quality_file = os.path.splitext(original_file)[0] + "_144p.mp4"

    print("Converting to 144p (ultra-low quality)...")
    # convert_quality(original_file, 144, ultra_low_quality_file)

    print("All conversions complete!")


Enter YouTube video URL: https://www.youtube.com/watch?v=EAE1TRSy3D0
Enter save directory (leave empty for current folder): 
[youtube] Extracting URL: https://www.youtube.com/watch?v=EAE1TRSy3D0
[youtube] EAE1TRSy3D0: Downloading webpage
[youtube] EAE1TRSy3D0: Downloading tv client config
[youtube] EAE1TRSy3D0: Downloading player 010fbc8d-main
[youtube] EAE1TRSy3D0: Downloading tv player API JSON
[youtube] EAE1TRSy3D0: Downloading ios player API JSON


[youtube] EAE1TRSy3D0: Downloading m3u8 information
[info] EAE1TRSy3D0: Downloading 1 format(s): 18
[download] Destination: ./How Nepali Villagers Live Closely with Nature ｜ Mountain Life IamSuman.mp4
[download] 100% of  124.82MiB in 00:00:06 at 19.41MiB/s  
Converting to 144p (ultra-low quality)...
All conversions complete!


In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install scenedetect

Note: you may need to restart the kernel to use updated packages.


### ab tak ka best

In [ ]:
import os
import subprocess
import joblib
import cv2
import torch
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, BlipProcessor, BlipForConditionalGeneration
from scenedetect import open_video, SceneManager
from scenedetect.detectors import ContentDetector
from sklearn.cluster import MiniBatchKMeans

# =========================
# Configuration (tweakable)
# =========================
VIDEO_PATH = "/content/How Nepali Villagers Live Closely with Nature ｜ Mountain Life IamSuman.mp4"
USER_PROMPT = "children sliding down a slope"
BEST_MOMENT_CLIP_PATH = f"{USER_PROMPT}.mp4"
THRESHOLD = 30.0
CACHE_DIR = "/kaggle/working/video_cache"
os.makedirs(CACHE_DIR, exist_ok=True)
video_filename = os.path.splitext(os.path.basename(VIDEO_PATH))[0]
CACHE_FILE_PATH = os.path.join(CACHE_DIR, f"{video_filename}_semantic_index_v2.pkl")

# Processing params
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" 
NUM_CLUSTERS = 8
TEMPORAL_WINDOW_SIZE = 3  # number of consecutive shots to treat as a moment
BATCH_SIZE = 16
USE_MIXED_PRECISION = True  # use torch.cuda.amp.autocast when available
PROMPT_EXPANSION = False  # set True to enable the expand_prompt() placeholder

# Safety / thresholds
MIN_SCENES_FOR_CLUSTERING = 2

# =========================
# Utilities
# =========================

def safe_device(device_str):
    if device_str == "cuda" and not torch.cuda.is_available():
        print("Warning: CUDA not available. Falling back to CPU.")
        return torch.device("cpu")
    return torch.device(device_str)


def run_ffmpeg_trim(input_path, output_path, start_time, end_time):
    """Trim by timestamps using ffmpeg. This is more robust for VFR videos than frame-based trimming."""
    # Use -ss start -to end -i input -c copy for fast trimming. Re-encode if precise cuts needed.
    cmd = [
        "ffmpeg",
        "-y",
        "-ss",
        f"{start_time:.3f}",
        "-to",
        f"{end_time:.3f}",
        "-i",
        input_path,
        "-c",
        "copy",
        output_path,
    ]
    try:
        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"Successfully trimmed clip: {output_path} ({start_time:.2f}s -> {end_time:.2f}s)")
    except subprocess.CalledProcessError:
        # Fall back to OpenCV frame-based reconstruction if ffmpeg copy fails
        print("ffmpeg trim failed or not available. Falling back to frame-based reconstruction.")
        return False
    return True


# Simple prompt expansion placeholder
def expand_prompt(prompt):
    """Optional: expand the prompt into related phrases. This is a placeholder.
    For production you'd call a small LLM (local/remote) to produce paraphrases.
    """
    if not PROMPT_EXPANSION:
        return [prompt]
    # naive example (very short) — replace with LLM in production
    expansions = [prompt]
    lower = prompt.lower()
    if "cook" in lower or "cooking" in lower:
        expansions += ["person cooking", "man preparing food", "woman cooking", "person preparing a meal"]
    return list(dict.fromkeys(expansions))


# =========================
# Scene detection
# =========================

def detect_scenes(video_path):
    print("--- Detecting shot boundaries with PySceneDetect ---")
    video = open_video(video_path)
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=THRESHOLD))
    try:
        scene_manager.detect_scenes(video, show_progress=False)
        scene_list = scene_manager.get_scene_list()
    except Exception as e:
        print("Scene detection failed:", e)
        scene_list = []

    if not scene_list:
        # fallback: treat whole video as a single scene using frame indices
        print("No scenes detected. Falling back to single-scene approach.")
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()
        return [(0, max(0, total_frames - 1))]

    scenes = [(s[0].get_frames(), s[1].get_frames()) for s in scene_list]
    print(f"Detected {len(scenes)} scenes.")
    return scenes


# =========================
# Representative frame reader
# =========================

def get_frame_by_index(video_path, frame_index):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        return None
    return Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))


# =========================
# Build semantic index (one-time)
# =========================

def extract_and_cache_data(video_path, scenes, device):
    print("--- Building semantic index (one-time) ---")

    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

    # Representative frame for each scene: middle frame value
    rep_indices = [s[0] + (s[1] - s[0]) // 2 for s in scenes]

    shot_data = []
    # Process in batches with careful indexing in case some frames fail to load
    for start in range(0, len(rep_indices), BATCH_SIZE):
        batch_indices = rep_indices[start:start + BATCH_SIZE]
        #getting the exact frame from by using the middle index
        batch_frames = [get_frame_by_index(video_path, idx) for idx in batch_indices]

        # Keep pairs of (index, image) where image succeeded
        valid_pairs = [(idx, img) for idx, img in zip(batch_indices, batch_frames) if img is not None]
        if not valid_pairs:
            print(f"  Skipping batch {start // BATCH_SIZE + 1}: no valid frames")
            continue

        valid_indices, valid_images = zip(*valid_pairs)

        # CLIP image embeddings
        clip_inputs = clip_processor(images=list(valid_images), return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            if USE_MIXED_PRECISION and device.type == 'cuda':
                with torch.cuda.amp.autocast():
                    image_features = clip_model.get_image_features(**clip_inputs)
            else:
                image_features = clip_model.get_image_features(**clip_inputs)
        image_features = image_features.cpu().numpy()

        # BLIP captions (batch)
        blip_inputs = blip_processor(images=list(valid_images), return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            if USE_MIXED_PRECISION and device.type == 'cuda':
                with torch.cuda.amp.autocast():
                    outs = blip_model.generate(**blip_inputs, max_new_tokens=50)
            else:
                outs = blip_model.generate(**blip_inputs, max_new_tokens=50)
        captions = blip_processor.batch_decode(outs, skip_special_tokens=True)

        # Store shot-level info (including normalized embeddings for cosine similarity)
        for j, idx in enumerate(valid_indices):
            emb = image_features[j]
            # normalize
            norm = np.linalg.norm(emb)
            if norm > 0:
                emb = emb / norm
            shot_data.append({
                'frame_index': int(idx),
                'cluster_id': None,
                'clip_embedding': emb.astype(np.float32),
                'caption': captions[j]
            })

        print(f"  Processed batch {start // BATCH_SIZE + 1} (valid frames: {len(valid_pairs)})")

    # Cluster with MiniBatchKMeans for speed; fall back if not enough shots
    embeddings = np.array([d['clip_embedding'] for d in shot_data]) if shot_data else np.zeros((0, 512))
    if len(embeddings) < MIN_SCENES_FOR_CLUSTERING:
        n_clusters = max(1, len(embeddings))
    else:
        n_clusters = min(NUM_CLUSTERS, len(embeddings))

    if len(embeddings) > 0:
        kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=0, batch_size=64, n_init=10)
        kmeans.fit(embeddings)
        labels = kmeans.labels_
        for i, d in enumerate(shot_data):
            d['cluster_id'] = int(labels[i])
    else:
        print("No embeddings were computed; skipping clustering.")

    # video metadata
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps if fps > 0 else 0
    cap.release()

    cache_data = {
        'video_meta': {'fps': fps, 'duration': duration, 'total_frames': total_frames},
        'scenes': scenes,
        'shot_data': shot_data
    }

    print(f"Saving semantic index to {CACHE_FILE_PATH}")
    joblib.dump(cache_data, CACHE_FILE_PATH)
    return cache_data


# =========================
# Search & reconstruct
# =========================

def search_video(cache_data, text_prompt, video_path, device):
    print("--- Searching semantic index ---")

    shot_data = cache_data.get('shot_data', [])
    if not shot_data:
        print("No shot data available in cache.")
        return

    # Expand prompt into paraphrases (optional)
    prompts = expand_prompt(text_prompt)

    # Load CLIP for text embeddings
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

    # Compute normalized text embedding for each expanded prompt and take max similarity across expansions
    prompt_embeddings = []
    for p in prompts:
        inputs = clip_processor(text=[p], return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            if USE_MIXED_PRECISION and device.type == 'cuda':
                with torch.cuda.amp.autocast():
                    t_emb = clip_model.get_text_features(**inputs)
            else:
                t_emb = clip_model.get_text_features(**inputs)
        t_emb = t_emb.cpu().numpy()
        # normalize
        t_emb = t_emb / (np.linalg.norm(t_emb, axis=1, keepdims=True) + 1e-10)
        prompt_embeddings.append(t_emb[0])

    # Stack shot embeddings
    shot_embeddings = np.array([d['clip_embedding'] for d in shot_data])

    # Compute similarity matrix between each prompt expansion and all shots, take max across expansions
    all_similarities = []
    for p_emb in prompt_embeddings:
        sims = shot_embeddings.dot(p_emb)  # already normalized -> cosine similarity
        all_similarities.append(sims)
    similarities = np.max(np.stack(all_similarities, axis=0), axis=0)

    # Attach scores
    for i, d in enumerate(shot_data):
        d['score'] = float(similarities[i])

    # Find best temporal window (sliding) — allow different aggregation strategies
    best_score = -999
    best_start = -1
    best_agg = 'mean'  # could be 'max' or weighted

    for i in range(0, len(shot_data) - TEMPORAL_WINDOW_SIZE + 1):
        window = shot_data[i: i + TEMPORAL_WINDOW_SIZE]
        scores = [w['score'] for w in window]
        if best_agg == 'mean':
            agg_score = float(np.mean(scores))
        elif best_agg == 'max':
            agg_score = float(np.max(scores))
        else:
            agg_score = float(np.mean(scores))

        if agg_score > best_score:
            best_score = agg_score
            best_start = i

    if best_start == -1:
        print("No suitable window found.")
        return

    print(f"Best window starts at shot {best_start} with score {best_score:.4f}")

    # Compute start/end times (seconds) using scene boundaries to trim cleanly
    scenes = cache_data['scenes']
    start_scene = scenes[best_start][0]
    end_scene = scenes[min(best_start + TEMPORAL_WINDOW_SIZE - 1, len(scenes) - 1)][1]

    fps = cache_data['video_meta']['fps']
    start_time = start_scene / fps
    end_time = end_scene / fps

    # Use ffmpeg for trimming (preferred). If it fails, fall back to frame-based reconstruct
    ffmpeg_ok = run_ffmpeg_trim(video_path, BEST_MOMENT_CLIP_PATH, start_time, end_time)
    if not ffmpeg_ok:
        reconstruct_clip_frame_based(video_path, BEST_MOMENT_CLIP_PATH, start_scene, end_scene, fps)

    # Print top-k shots
    TOP_K = 10
    shot_data_sorted = sorted(shot_data, key=lambda x: x['score'], reverse=True)
    print(f"\nTop {TOP_K} matched shots:")
    for i, d in enumerate(shot_data_sorted[:TOP_K]):
        ts = d['frame_index'] / fps
        print(f" Rank {i+1}: time={ts:.2f}s frame={d['frame_index']} score={d['score']:.4f} caption=\"{d['caption']}\"")


# Fallback frame-based reconstruction if ffmpeg not available / fails
def reconstruct_clip_frame_based(video_path, output_path, start_frame, end_frame, fps):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video for frame-based reconstruction: {video_path}")
        return
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    current = start_frame
    while current <= end_frame:
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
        current += 1

    cap.release()
    out.release()
    print(f"Saved frame-based clip to {output_path}")


# =========================
# Main
# =========================

def main(user_prompt):
    device = safe_device(DEVICE)

    if not os.path.exists(VIDEO_PATH):
        print(f"Video not found: {VIDEO_PATH}")
        return

    if os.path.exists(CACHE_FILE_PATH):
        print(f"Loading cached semantic index from {CACHE_FILE_PATH}")
        cache_data = joblib.load(CACHE_FILE_PATH)
    else:
        scenes = detect_scenes(VIDEO_PATH)
        cache_data = extract_and_cache_data(VIDEO_PATH, scenes, device)

    search_video(cache_data, user_prompt, VIDEO_PATH, device)


if __name__ == '__main__':
    # Example usage

    main(USER_PROMPT)


--- Detecting shot boundaries with PySceneDetect ---


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Detected 56 scenes.
--- Building semantic index (one-time) ---


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

C:\Users\Owais\AppData\Local\Temp\ipykernel_25120\618556656.py:167: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
C:\Users\Owais\AppData\Local\Temp\ipykernel_25120\618556656.py:177: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


...

Saving semantic index to ./surveilens_cluster/video_cache\test_video_semantic_index_v2.pkl
--- Searching semantic index ---


C:\Users\Owais\AppData\Local\Temp\ipykernel_25120\618556656.py:258: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Best window starts at shot 46 with score 0.3145


FileNotFoundError: [WinError 2] The system cannot find the file specified

c:\Users\Owais\anaconda3\envs\cuda-env-torch\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Owais\.cache\huggingface\hub\models--Salesforce--blip-image-captioning-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


# scene detect best working model


In [11]:
import cv2
import torch
import numpy as np
from PIL import Image
import os
from torchvision import transforms
from sklearn.cluster import KMeans
from transformers import BlipProcessor, BlipForConditionalGeneration, CLIPProcessor, CLIPModel
from sentence_transformers import SentenceTransformer, util
import joblib
from scenedetect import open_video, SceneManager
from scenedetect.detectors import ContentDetector

# --- Configuration ---
# You can adjust these parameters
VIDEO_PATH = "/content/How Nepali Villagers Live Closely with Nature ｜ Mountain Life IamSuman.mp4"  # Path to your input video
SCENE_CLIP_PATH = "reconstructed_scene_clip.mp4" # Path for the full scene clip

USER_PROMPT = "kids sliding down a slope"
BEST_MOMENT_CLIP_PATH = f"{USER_PROMPT}_best_moment_clip.mp4" # Path for the 10-sec highlight clip

DEVICE = "cuda" # Options: "cuda" or "cpu"

# --- Model & Algorithm Parameters ---
NUM_CLUSTERS = 8 # Fixed number of clusters for scene grouping. Faster than Elbow method.
TEMPORAL_WINDOW_SIZE = 3 # Number of consecutive shots to consider as a single "moment".
TOP_K_FRAMES_TO_DISPLAY = 10 # The number of top frames to list at the end.

# --- Cache File Paths ---
CACHE_DIR = "/kaggle/working/video_cache"
os.makedirs(CACHE_DIR, exist_ok=True)
video_filename = os.path.splitext(os.path.basename(VIDEO_PATH))[0]
CACHE_FILE_PATH = os.path.join(CACHE_DIR, f"{video_filename}_semantic_index.pkl")

def detect_scenes(video_path):
    """
    Step 1: Use PySceneDetect to find shot boundaries.
    Returns a list of tuples, where each tuple is (start_frame, end_frame).
    """
    print("--- Step 1: Detecting Shot Boundaries ---")
    video = open_video(video_path)
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector())
    scene_manager.detect_scenes(video, show_progress=True)
    scene_list = scene_manager.get_scene_list()

    # Convert to a simple list of (start, end) frame numbers
    scenes = [(s[0].get_frames(), s[1].get_frames()) for s in scene_list]
    print(f"Detected {len(scenes)} scenes/shots.")
    return scenes

def get_representative_frame_by_index(video_path, frame_index):
    """
    Memory-efficiently reads a single frame from a video file by its index.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened(): return None
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ret, frame = cap.read()
    cap.release()
    if not ret: return None
    return Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

def extract_and_cache_data(video_path, scenes, device):
    """
    Step 2: Extracts features, gets captions, and caches everything.
    This is the main one-time processing function.
    """
    print("\n--- Step 2: Building Semantic Index (One-Time Process) ---")

    # --- Load Models ---
    print("Loading AI models...")
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    caption_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    caption_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

    # --- Select Representative Frames ---
    # We pick the middle frame of each detected shot as its representative.
    representative_frame_indices = [s[0] + (s[1] - s[0]) // 2 for s in scenes]

    # --- Batch Processing for Efficiency ---
    batch_size = 16
    all_shot_data = []

    print(f"Processing {len(representative_frame_indices)} representative frames in batches...")
    for i in range(0, len(representative_frame_indices), batch_size):
        batch_indices = representative_frame_indices[i:i+batch_size]
        batch_images = [get_representative_frame_by_index(video_path, idx) for idx in batch_indices]

        # Filter out any frames that failed to load
        valid_images = [img for img in batch_images if img is not None]
        if not valid_images: continue

        # --- Get CLIP Embeddings in Batch ---
        inputs = clip_processor(images=valid_images, return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            image_features = clip_model.get_image_features(**inputs).cpu().numpy()

        # --- Get Captions in Batch ---
        inputs = caption_processor(images=valid_images, return_tensors="pt").to(device)
        with torch.no_grad():
            out = caption_model.generate(**inputs, max_new_tokens=50)
        captions = caption_processor.batch_decode(out, skip_special_tokens=True)

        # Store data for this batch
        for j, frame_idx in enumerate(batch_indices):
            all_shot_data.append({
                'frame_index': frame_idx,
                'clip_embedding': image_features[j],
                'caption': captions[j]
            })
        print(f"  Processed batch {i//batch_size + 1}/{len(representative_frame_indices)//batch_size + 1}")

    # --- Cluster Shots into Semantic Groups ---
    print("\nClustering shots into semantic groups...")
    shot_embeddings = np.array([d['clip_embedding'] for d in all_shot_data])
    kmeans = KMeans(n_clusters=min(NUM_CLUSTERS, len(shot_embeddings)), random_state=0, n_init=10).fit(shot_embeddings)

    for i, data in enumerate(all_shot_data):
        data['cluster_id'] = kmeans.labels_[i]

    # --- Cache everything ---
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / fps
    cap.release()

    cache_data = {
        'video_meta': {'fps': fps, 'duration': duration},
        'scenes': scenes,
        'shot_data': all_shot_data
    }

    print(f"\nSaving semantic index to {CACHE_FILE_PATH}")
    joblib.dump(cache_data, CACHE_FILE_PATH)

    return cache_data

def search_video(cache_data, text_prompt, device):
    """
    Step 3 & 4: Searches the cached index and reconstructs the best clip.
    """
    print("\n--- Step 3: Searching for Best Temporal Moment ---")

    # --- Load Models ---
    similarity_model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

    # --- Encode Prompt ---
    # Advanced option (5): Expand prompt here with an LLM before encoding.
    prompt_embedding = similarity_model.encode(text_prompt, convert_to_tensor=True)

    # --- Calculate Similarity Scores ---
    shot_data = cache_data['shot_data']
    captions = [d['caption'] for d in shot_data]
    caption_embeddings = similarity_model.encode(captions, convert_to_tensor=True)

    similarities = util.pytorch_cos_sim(prompt_embedding, caption_embeddings).squeeze().cpu().numpy()

    for i, data in enumerate(shot_data):
        data['score'] = similarities[i]

    # --- Find Best Temporal Window ---
    best_window_score = -1
    best_window_start_index = -1

    for i in range(len(shot_data) - TEMPORAL_WINDOW_SIZE + 1):
        window = shot_data[i : i + TEMPORAL_WINDOW_SIZE]
        # Calculate the average score of the window
        window_score = np.mean([d['score'] for d in window])

        if window_score > best_window_score:
            best_window_score = window_score
            best_window_start_index = i

    if best_window_start_index == -1:
        print("Could not find a suitable match in the video.")
        return

    print(f"Best temporal window starts at shot {best_window_start_index} with an average score of {best_window_score:.4f}")

    # --- Step 4: Reconstruct Clip & Display Results ---
    best_window_shots = shot_data[best_window_start_index : best_window_start_index + TEMPORAL_WINDOW_SIZE]
    scenes = cache_data['scenes']

    # Use shot boundaries for clean trimming (8)
    start_frame = scenes[best_window_start_index][0]
    end_frame = scenes[best_window_start_index + TEMPORAL_WINDOW_SIZE - 1][1]

    fps = cache_data['video_meta']['fps']
    start_time = start_frame / fps
    end_time = end_frame / fps

    print(f"\nReconstructing highlight clip from {start_time:.2f}s to {end_time:.2f}s.")
    reconstruct_clip(VIDEO_PATH, BEST_MOMENT_CLIP_PATH, start_frame, end_frame, fps)

    # --- Display top K matched frames from the entire video ---
    print(f"\n--- Top {TOP_K_FRAMES_TO_DISPLAY} Best Matched Shots from the Video ---")
    shot_data.sort(key=lambda x: x['score'], reverse=True)

    for i, data in enumerate(shot_data[:TOP_K_FRAMES_TO_DISPLAY]):
        ts = data['frame_index'] / fps
        print(f"  Rank {i+1}:")
        print(f"    - Timestamp: {ts:.2f}s (Frame: {data['frame_index']})")
        print(f"    - Score: {data['score']:.4f}")
        print(f"    - Caption: '{data['caption']}'")
        # Advanced option (6): Run object detection here and show if key nouns are present.

def reconstruct_clip(video_path, output_path, start_frame, end_frame, fps):
    """Helper function to write a video clip from a range of frames."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video for reconstruction at {video_path}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    current_frame = start_frame
    while current_frame <= end_frame:
        ret, frame = cap.read()
        if not ret: break
        out.write(frame)
        current_frame += 1

    cap.release()
    out.release()
    print(f"Successfully saved clip to '{output_path}'")

def main():
    """Main function to run the entire pipeline."""
    if not os.path.exists(VIDEO_PATH):
        print(f"FATAL ERROR: Input video not found at '{VIDEO_PATH}'")
        return

    # Set up device, with a fallback to CPU if CUDA is not available
    if DEVICE == "cuda" and not torch.cuda.is_available():
        print("Warning: CUDA not available, falling back to CPU. This will be slow.")
        device = "cpu"
    else:
        device = DEVICE
    print(f"--- Using device: {device.upper()} ---")

    # Check if a pre-computed index exists
    if os.path.exists(CACHE_FILE_PATH):
        print(f"--- Found pre-computed index. Loading from {CACHE_FILE_PATH} ---")
        cache_data = joblib.load(CACHE_FILE_PATH)
        print("Loading complete.")
    else:
        print("--- No index found. Processing video from scratch... ---")
        scenes = detect_scenes(VIDEO_PATH)
        cache_data = extract_and_cache_data(VIDEO_PATH, scenes, device)

    # Search the video using the (newly created or loaded) index
    search_video(cache_data, USER_PROMPT, device)

if __name__ == "__main__":
    main()


--- Using device: CUDA ---
--- No index found. Processing video from scratch... ---
--- Step 1: Detecting Shot Boundaries ---


  Detected: 0 | Progress:   0%|          | 0/49796 [00:00<?, ?frames/s]INFO:pyscenedetect:Detecting scenes...
  Detected: 250 | Progress: 100%|██████████| 49796/49796 [01:17<00:00, 642.96frames/s]


Detected 251 scenes/shots.

--- Step 2: Building Semantic Index (One-Time Process) ---
Loading AI models...
Processing 251 representative frames in batches...
  Processed batch 1/16
  Processed batch 2/16
  Processed batch 3/16
  Processed batch 4/16
  Processed batch 5/16
  Processed batch 6/16
  Processed batch 7/16
  Processed batch 8/16
  Processed batch 9/16
  Processed batch 10/16
  Processed batch 11/16
  Processed batch 12/16
  Processed batch 13/16
  Processed batch 14/16
  Processed batch 15/16
  Processed batch 16/16

Clustering shots into semantic groups...

Saving semantic index to /kaggle/working/video_cache/How Nepali Villagers Live Closely with Nature ｜ Mountain Life IamSuman_semantic_index.pkl

--- Step 3: Searching for Best Temporal Moment ---


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Best temporal window starts at shot 33 with an average score of 0.4220

Reconstructing highlight clip from 209.90s to 229.90s.
Successfully saved clip to 'kids sliding down a slope_best_moment_clip.mp4'

--- Top 10 Best Matched Shots from the Video ---
  Rank 1:
    - Timestamp: 212.63s (Frame: 6379)
    - Score: 0.5550
    - Caption: 'a group of children walking down a dirt road'
  Rank 2:
    - Timestamp: 1061.40s (Frame: 31842)
    - Score: 0.4615
    - Caption: 'a group of people walking down a hill'
  Rank 3:
    - Timestamp: 218.70s (Frame: 6561)
    - Score: 0.4545
    - Caption: 'a group of children sitting on the ground'
  Rank 4:
    - Timestamp: 774.43s (Frame: 23233)
    - Score: 0.4461
    - Caption: 'a group of people are walking down a hill'
  Rank 5:
    - Timestamp: 36.67s (Frame: 1100)
    - Score: 0.3803
    - Caption: 'a group of people are climbing up a hill'
  Rank 6:
    - Timestamp: 446.20s (Frame: 13386)
    - Score: 0.3652
    - Caption: 'a group of black bear